# Tutorial de Big Data (UNT) 2024
## Tutorial 9 - Cross-validation

**Objetivo:** 
Que se familiaricen con la técnica de K-fold Cross Validation


In [119]:
import pandas as pd
import numpy as np
#from ISLP import load_data

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Vamos a trabajar con la base 'Auto' de ISLP.

Tiene información para 392 vehículos. Kilometraje de gasolina, caballos de fuerza El dataset tiene las siguiente variables:
- mpg: millas por galón
- cylinders: Número de cilindros entre 4 y 8
- displacement: Cilindrada o desplazamiento del motor (pulgadas cúbicas)
- horsepower: Caballos del motor
- weight: Peso del vehículo (libras)
- acceleration: Tiempo de aceleración de 0 a 100 km/h (seg.)
- year: Año del modelo (módulo 100)
- origin: Origen del vehículo (1. Americano, 2. Europeo, 3. Japonés)
- name: Nombre del vehículo

En este [link](https://islp.readthedocs.io/en/latest/datasets/Auto.html) tienen más información

In [ ]:
auto = pd.read_csv("Auto.csv")

# Dimensión de la base
print("Dimensión del dataframe:", auto.shape)

# Variables e información
#print(auto.dtypes)
print(auto.info())

auto.head()

In [ ]:
# Hay duplicados?
print("Duplicados:", auto.duplicated().sum())

# Hay valores faltantes?
print("\n Missings:\n", auto.isnull().sum()) # conteo
#print(auto.isnull().mean() * 100) # como porcentaje

# No hay duplicados ni missing values

In [ ]:
# Inspección rápida de las variables y sus valores
auto.describe()

In [ ]:
auto["origin"].value_counts()

In [ ]:
# Notamos que origin es una variable categórica (toma valores 1, 2, 3)

# Usaremos one-hot encoding para transformar la columna categórica llamada origin 
# en varias columnas binarias (dummies).
# Cómo? get_dummies 
origin_dummies = pd.get_dummies(auto['origin'], prefix='origin')

# Concatenamos con el df original
auto_d = pd.concat([auto, origin_dummies], axis=1)
auto_d.tail()


Ahora vamos a trabajar con mpg como variable dependiente y horsepower como independiente

In [125]:
# Guardo los vectores de variable dependiente y de variable independiente respectivamente:
y = auto_d['mpg']
X = auto_d['horsepower']
X = np.array(X).reshape((-1, 1))

# Parto la base en dos y transformo el vector x: 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)


In [ ]:
# Regresión lineal
lreg=LinearRegression()
lreg.fit(x_train,y_train)
y_pred_lreg=lreg.predict(x_test)

print("R2:", r2_score(y_test,y_pred_lreg))

print("Coeficiente:", lreg.coef_)
y_pred_lreg = lreg.predict(x_test)
ecm_lreg = mean_squared_error(y_test, y_pred_lreg)
print('Error cuadrático medio (test):', ecm_lreg)

In [ ]:
# Recordemos las Regresiones Polinómicas:
# Implican una transformación polinómica de las X, para luego implementar la regresión

# Veamos un modelo cuadrático:
poly = PolynomialFeatures(degree = 2, include_bias=False) 
# Recordar setear include_bias=False dado que en la regresión lineal -con LinearRegression- se incluirá la columna de 1s

#print(x_train)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)  
np.set_printoptions(suppress = True) # evita que el print salga con notación científica
print('X luego de la transformación:\n', x_train_poly[:5,])

# Ajustamos el modelo
model = LinearRegression().fit(x_train_poly, y_train) 
print('\nIntercepto:', model.intercept_)
print('Coeficientes:', model.coef_)

# Calculamos el Error Cuadrático Medio
y_pred_poly = model.predict(x_test_poly)
ecm2 = mean_squared_error(y_test, y_pred_poly)
print('Error cuadrático medio (test):', ecm2)

In [ ]:
# Veamos un modelo cúbico:
poly = PolynomialFeatures(degree = 3, include_bias=False) 

x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)  
  
model = LinearRegression().fit(x_train_poly, y_train) 
y_pred_poly = model.predict(x_test_poly)

ecm3 = mean_squared_error(y_test, y_pred_poly)

print('\nIntercepto:', model.intercept_)
print('Coeficientes:', model.coef_)
print('Error cuadrático medio (test):', ecm3)

A priori, viendo el ECM, parecería que la regresión polinomial de grado 2 es la que mejor funciona 

In [ ]:
# Creamos un nuevo vector de X y aplicamos las transformaciones
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1) 
# Valores entre el minimo y el maximo de X. 
# linspace por default crea 50 valores
# Aplicamos las transformaciones polinomicas
X_seq_poly = poly.fit_transform(X_seq)  

# Gráfico
plt.figure()
plt.scatter(x_train, y_train)
plt.plot(X_seq, model.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree 3")
plt.show()

In [ ]:
# Ahora supongamos que cambiamos la muestra y repetimos, hacemos lo mismo otra vez
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 50)

# Qué error esperarían que obtengamos esta vez?
poly = PolynomialFeatures(degree = 2, include_bias=False) 

x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)  
  
model = LinearRegression().fit(x_train_poly, y_train) 
y_pred_poly = model.predict(x_test_poly)

ecm3b = mean_squared_error(y_test, y_pred_poly)

print('\nIntercepto:', model.intercept_)
print('Coeficientes:', model.coef_)
print('Error cuadrático medio (test):', ecm3b)

In [135]:
# Cómo podemos repetir el código sin escribirlo por tercera vez?
# Podemos hacer que nuestro código funcione para otros grados?

def transf_reg_poly(grado, x_train, x_test, y_train, y_test):
    '''
    La función realiza una transformación polinomial y luego corre una regresión lineal polinómica
    Input:
        grado
        x_train, x_test, y_train, y_test
    Output:
        modelo, ecm
    '''
    poly = PolynomialFeatures(degree = grado, include_bias=False) 

    x_train_poly = poly.fit_transform(x_train)
    x_test_poly = poly.fit_transform(x_test)  
  
    model = LinearRegression().fit(x_train_poly, y_train) 
    y_pred_poly = model.predict(x_test_poly)
    
    ecm = mean_squared_error(y_test, y_pred_poly)
    return model, ecm


In [91]:
ecm1 = transf_reg_poly(1, x_train, x_test, y_train, y_test)[1]
ecm2b = transf_reg_poly(2, x_train, x_test, y_train, y_test)[1]
ecm3b = transf_reg_poly(3, x_train, x_test, y_train, y_test)[1]
ecm4b = transf_reg_poly(4, x_train, x_test, y_train, y_test)[1]
ecm5b = transf_reg_poly(5, x_train, x_test, y_train, y_test)[1]

In [ ]:
print('Regresión lineal:', ecm_lreg)
print('Grado2:', ecm2)
print('Grado3:', ecm3)

print('\nRegresión lineal:', ecm1)
print('Grado2:', ecm2b)
print('Grado3:', ecm3b)
print('Grado4:', ecm4b)
print('Grado5:', ecm5b)

Podemos concluir que la regresión lineal funciona peor, y que introducir un término cuadrático reduce el ECM. Pero en el caso de introducir un término cúbico, no es obvio si funciona mejor o no...

El ECM puede variar según qué observaciones quedaron incluidas en los sets de train y test

###  Nuevo enfoque: K-FOLD CROSS-VALIDATION  

Es un **técnica de remuestreo**. Se usa para estimar el error (test) asociado a un método de aprendizaje, para:  
- Elegir el nivel de complejidad optimo (Model selection)
- Evaluar el error de pronóstico fuera de la muestra (futura, condicional, contra fáctica, etc.) (Model Assesment)

Consiste en:
- Dividir las observaciones en k folds (pliegues), del mismo tamaño, aleatoriamente. 
- Ajustar el modelo k veces, cada vez con k-1 folds (distintos cada vez). Computar k veces el error de predicción en el fold reservado. (cada fold se usa k-1 veces como training set y 1 vez como test set).
- Estimar el error de predicción, estimación que surge de promediar las K estimaciones obtenidas.

Vamos a usar [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) de Scikitlearn

In [ ]:
from IPython.display import Image, display

display(Image(url="https://global.discourse-cdn.com/dlai/optimized/3X/a/3/a3ed2de61c2b4fa00f1b7e939753e1a7e181afb0_2_690x476.png"
))

In [ ]:
y = auto['mpg']
X = auto['horsepower']
X = np.array(X).reshape((-1, 1))

from sklearn.model_selection import KFold

ecms = pd.DataFrame(columns=["grado", "particion", "ecm"])
ecms

Lo usual es usar K=5 o K=10

In [ ]:
print(type(X), type(y))
print(X.shape, y.shape)
#print(X.flatten(), y)

In [ ]:
K = 10

for grado in range(2, 10):   

    kf = KFold(n_splits=K, shuffle=True, random_state=100)
    
    # El método kf.split aplicado a X nos da los conjuntos de índices que necesitamos para
    # partir nuestros conjunto de datos en training y testing en cada iteración.
    #  OXXXX
    #  XOXXX
    #  XXOXX
    #  XXXOX
    #  XXXXO
    
    for i, (train_index, val_index) in enumerate(kf.split(X)):   
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        #print(i, x_train.shape[0])
        
        ecm = transf_reg_poly(grado, x_train, x_val, y_train, y_val)[1]
            
        df_i = pd.DataFrame({"grado": grado, "particion": i, "ecm": ecm}, index=[0])
        ecms = pd.concat([ecms, df_i])
    
ecms = ecms.astype({"grado":int, "particion":int})
ecms

Una corrección para no contaminar los datos...

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 50)
print(type(X_train), type(y_train))
print(X_train.shape, y_train.shape)
#print(X_train, y_train)

In [ ]:
ecms = pd.DataFrame(columns=["grado", "particion", "ecm"])
ecms

K = 5

for grado in range(1, 10):   

    kf = KFold(n_splits=K, shuffle=True, random_state=100)
    
    # El método kf.split aplicado a X nos da los conjuntos de índices que necesitamos para
    # partir nuestros conjunto de datos en training y testing en cada iteración.
    #  OXXXX
    #  XOXXX
    #  XXOXX
    #  XXXOX
    #  XXXXO
    
    for i, (train_index2, val_index2) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train[train_index2], X_train[val_index2]
        y_train_fold, y_val_fold = y_train.iloc[train_index2], y_train.iloc[val_index2]
        #print(i, X_train_fold.shape[0])
        
        ecm = transf_reg_poly(grado, X_train_fold, X_val_fold, y_train_fold, y_val_fold)[1]
            
        df_i = pd.DataFrame({"grado": grado, "particion": i, "ecm": ecm}, index=[0])
        ecms = pd.concat([ecms, df_i])
    
ecms = ecms.astype({"grado":int, "particion":int})
ecms

#### Cómo elegir el modelo?

In [ ]:
# Una opción: visualizar los ECMs en un boxplot
import seaborn as sns
sns.set()
ss = sns.boxplot(data=ecms, x="grado", y="ecm")


In [ ]:
# Una opción para ver el mejor modelo sería sacar el error promedio para cada grado:
ecms_avg = ecms.groupby('grado').agg({'ecm':'mean'})
ecms_avg.reset_index(inplace = True)
ecms_avg.astype({"grado":int})
ecms_avg

In [ ]:
# Función para seleccionar 
min_ecm = np.Inf
grado = None

for index, row in ecms_avg.iterrows():
    if row['ecm'] < min_ecm:
        min_ecm = row['ecm']
        grado = row['grado'].astype(int)

print('El mínimo error es', round(min_ecm, 2), 'y se da con un polinomio de grado', grado)

In [ ]:
# Finalmente construimos el modelo polinomial de grado 2 y lo graficamos 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 50)

modelo = transf_reg_poly(grado, x_train, x_test, y_train, y_test)[0]
ecm = transf_reg_poly(grado, x_train, x_test, y_train, y_test)[1]
        
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
poly = PolynomialFeatures(degree = grado, include_bias=False) 
X_seq_poly = poly.fit_transform(X_seq)  

x_test_poly = poly.fit_transform(x_test)  
y_pred_poly = modelo.predict(x_test_poly)
ecm_final = mean_squared_error(y_test, y_pred_poly)

plt.figure()
plt.scatter(x_train, y_train)
plt.plot(X_seq, modelo.predict(X_seq_poly),color="black")
plt.title("Polynomial regression with degree {}".format(grado))
plt.show()

ecm_final